In [1]:
import pyspark

In [2]:
sc = pyspark.SparkContext(appName = "Oi mundo!")

In [3]:
sc

<SparkContext master=local[*] appName=Oi mundo!>

In [4]:
rdd = sc.sequenceFile('part-00000')

In [5]:
rdd.count()

36133

In [6]:
result = rdd.take(10)

In [7]:
palavra1 = "playstation"
palavra2 = "nintendo"

In [8]:
def conta_palavras(item):
    url, conteudo = item
    palavras = conteudo.strip().split()
    palavras = [
        i.lower()
        for i in palavras
        if (any(j.isalpha() for j in i) and len(i) > 3)
    ]

    
    if numero == 0:
        if palavra1 in palavras and palavra2 in palavras:
            return [(i, 1) for i in palavras]
    elif numero == 1:
        if palavra1 in palavras and palavra2 not in palavras:
            return [(i, 1) for i in palavras]
    elif numero == 2:
        if palavra1 not in palavras and palavra2 in palavras:
            return [(i, 1) for i in palavras]
    return []

def conta_doc(item):
    url, conteudo = item
    palavras = conteudo.strip().split()
    palavras = [
        i.lower()
        for i in palavras
        if (any(j.isalpha() for j in i) and len(i) > 3)
    ]
    if numero == 0:
        if palavra1 in palavras and palavra2 in palavras:
            return [(i, 1) for i in set(palavras)]
    elif numero == 1:
        if palavra1 in palavras and palavra2 not in palavras:
            return [(i, 1) for i in set(palavras)]
    elif numero == 2:
        if palavra1 not in palavras and palavra2 in palavras:
            return [(i, 1) for i in set(palavras)]
    return []

def junta_contagens(nova_contagem, contagem_atual):
    return nova_contagem + contagem_atual


numero = 0
rdd_doc_freq = rdd.flatMap(conta_doc).reduceByKey(junta_contagens)
rdd_palavra_freq = rdd.flatMap(conta_palavras).reduceByKey(junta_contagens)

numero = 1
rdd_doc_freq1 = rdd.flatMap(conta_doc).reduceByKey(junta_contagens)
rdd_palavra_freq1 = rdd.flatMap(conta_palavras).reduceByKey(junta_contagens)

numero = 2
rdd_doc_freq2 = rdd.flatMap(conta_doc).reduceByKey(junta_contagens)
rdd_palavra_freq2 = rdd.flatMap(conta_palavras).reduceByKey(junta_contagens)


In [9]:
rdd_doc_freq.take(10)

[('feminino', 19),
 ('produtos', 100),
 ('criativos', 3),
 ('series', 39),
 ('placas', 44),
 ('qua15467', 1),
 ('series,', 1),
 ('crianças', 19),
 ('vazio', 31),
 ('extração', 3)]

In [10]:
N = rdd.count()
DOC_COUNT_MIN = 5
DOC_COUNT_MAX = 0.7*N

def filtra_doc_freq(item):
    contagem = item[1]
    return (contagem < DOC_COUNT_MAX) and (contagem >= DOC_COUNT_MIN)

rdd_doc_freq_filtrado = rdd_doc_freq.filter(filtra_doc_freq)
rdd_doc_freq_filtrado1 = rdd_doc_freq1.filter(filtra_doc_freq)
rdd_doc_freq_filtrado2 = rdd_doc_freq2.filter(filtra_doc_freq)

In [11]:
import math

def computa_idf(item):
    palavra, contagem = item
    idf = math.log10(N/contagem)
    return (palavra, idf)

def computa_freq(item):
    palavra, contagem = item
    freq = math.log10(1 + contagem)
    return (palavra,freq)


def computa_rele(item):
    palavra, contagem = item
    freq, idf = contagem
    relevancia = freq*idf
    return (palavra, relevancia)

In [12]:
import csv

rdd_idf = rdd_doc_freq_filtrado.map(computa_idf)
rdd_freq = rdd_palavra_freq.map(computa_freq)
rdd_rele = rdd_idf.join(rdd_freq).map(computa_rele)

top_100 = rdd_rele.takeOrdered(100, key=(lambda x: -x[1]))

with open("games.csv", "w") as games:
    csv0 = csv.writer(games)
    csv0.writerow(["palavra", "freq"])
    for row in top_100:
        csv0.writerow(row)
        
        
rdd_idf1 = rdd_doc_freq_filtrado1.map(computa_idf)
rdd_freq1 = rdd_palavra_freq1.map(computa_freq)
rdd_rele1 = rdd_idf1.join(rdd_freq1).map(computa_rele)

top_100_1 = rdd_rele1.takeOrdered(100, key=(lambda x: -x[1]))

with open("playstation.csv", "w") as playstation:
    csv1 = csv.writer(playstation)
    csv1.writerow(["palavra", "freq"])
    for row in top_100_1:
        csv1.writerow(row)
        
        
        
rdd_idf2 = rdd_doc_freq_filtrado2.map(computa_idf)
rdd_freq2 = rdd_palavra_freq2.map(computa_freq)
rdd_rele2 = rdd_idf2.join(rdd_freq2).map(computa_rele)

top_100_2 = rdd_rele2.takeOrdered(100, key=(lambda x: -x[1]))

with open("nintendo.csv", "w") as nintendo:
    csv2 = csv.writer(nintendo)
    csv2.writerow(["palavra", "freq"])
    for row in top_100_2:
        csv2.writerow(row)